# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm import trange, tqdm

импорт библиотек

In [ ]:
reqx = requests.get('https://tiraet.com/catalog/bytovaya/?PAGEN_1=102')

In [ ]:
soup = bs(reqx.text, 'html.parser')

In [ ]:
element = soup.findAll('div', class_='item_info')

In [ ]:
element

In [101]:
title = element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip()
title

'Кофеварка Adler AD 4404cr (молотый кофе, эспрессо/капучино, рожк, 850Вт, 15бар, 1.6л)'

создаем переменную с наименованием товара

In [102]:
urls = 'https://tiraet.com' + element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href']

In [103]:
counts = element[0].find('div', class_='item-stock js-show-stores js-show-info-block').text.strip()

In [104]:
art = int(element[0].find('div', class_='article_block').text.replace('Арт.: ',''))

In [105]:
price = float(element[7].find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ',''))

In [106]:
old_price = float(element[7].find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(element[7].find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else 0

In [107]:
datе = pd.to_datetime('today').normalize()

In [110]:
data = []
for i in element:
    data.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                 'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],  
                 'datе' : pd.to_datetime('today').normalize(),                
                 'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                 'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN    ,
                 'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN                
                })

In [120]:
data = pd.DataFrame(data)
data

,title,urls,datе,counts,id,price,old_price
0,"85' Телевизор LED Samsung UE85CU7100UXUA SmTV,...",https://tiraet.com/catalog/bytovaya/televizor-...,2024-06-05,Мало,83664,34999.0,NaN
1,Кондиционер Cooper&Hunter CHML-U36RK4 Outdoor ...,https://tiraet.com/catalog/bytovaya/klimatiche...,2024-06-05,Мало,80511,35599.1,NaN
2,Кондиционер Cooper&Hunter CHML-U42RK5 Outdoor ...,https://tiraet.com/catalog/bytovaya/klimatiche...,2024-06-05,Достаточно,77115,36998.9,NaN
3,"65' Телевизор OLED Samsung QE65S90CAUXUA SmTV,...",https://tiraet.com/catalog/bytovaya/televizor-...,2024-06-05,Мало,86148,35599.1,41998.9
4,"75' Телевизор QLED Samsung QE75Q80CAUXUA SmTV,...",https://tiraet.com/catalog/bytovaya/televizor-...,2024-06-05,Мало,81035,40498.9,45799.1
5,"85' Телевизор QLED Samsung QE85Q60CAUXUA SmTV,...",https://tiraet.com/catalog/bytovaya/televizor-...,2024-06-05,Мало,86142,36599.0,45999.0
6,"Электрочайник Braun Multiquick 3 WK 300 (1.7л,...",https://tiraet.com/catalog/bytovaya/tehnika-dl...,2024-06-05,Нет в наличии,77716,NaN,NaN
7,"Кофеварка Adler AD 4404cr (молотый кофе, эспре...",https://tiraet.com/catalog/bytovaya/tehnika-dl...,2024-06-05,Нет в наличии,75833,1558.0,NaN


In [15]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [16]:
url = ['https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=']

In [117]:
url = ['https://tiraet.com/catalog/bytovaya/?PAGEN_1=102'] 

In [72]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/uslugi/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',
         
]

In [111]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'urls', 'datе', 'counts', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'datе' : pd.to_datetime('today').normalize(),
                             'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

In [121]:
df = parsing_2(oll_url)
df

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [03:18<00:00,  2.48s/it]


,title,urls,datе,counts,id,price,old_price
0,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/ak...,2024-06-05,Много,284,5.0,NaN
1,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-...",https://tiraet.com/catalog/Komplektujuschie/ak...,2024-06-05,Много,8683,5.0,NaN
2,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм,...",https://tiraet.com/catalog/Komplektujuschie/ak...,2024-06-05,Много,85078,6.6,NaN
3,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROT...",https://tiraet.com/catalog/Komplektujuschie/ak...,2024-06-05,Много,12819,6.6,NaN
4,"Диск DVD+R VERBATIM 16X 4,7GB (1шт.) (43550/...",https://tiraet.com/catalog/Komplektujuschie/ak...,2024-06-05,Много,4975,7.4,NaN
...,...,...,...,...,...,...,...
11419,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Мало,27477,7920.0,NaN
11420,Узел термозакрепления (для сетевых апп)CANON i...,https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Мало,55309,8085.0,NaN
11421,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Мало,56144,8250.0,NaN
11422,Картридж EPSON MX-80/LX-300/400/800/850/880 Pa...,https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Нет в наличии,249,38.0,NaN


In [122]:
today = pd.to_datetime('today').normalize()

In [123]:
today = today.to_pydatetime()

создаем переменную, которая хранит информацию о текущей дате

In [124]:
df.to_csv('csv/tiraet_'+str(today.day)+'_'+str(today.month)+'_'+str(today.year)+'.csv', index=False)  

In [125]:
df[df['counts']=='Нет в наличии']

,title,urls,datе,counts,id,price,old_price
3622,"DVD+-RW/CD-RW LITEON IHAS124-04 DL, BLACK",https://tiraet.com/catalog/Komplektujuschie/ko...,2024-06-05,Нет в наличии,36716,278.8,NaN
4398,Освежитель воздуха автоматический Air Wick Lif...,https://tiraet.com/catalog/Kompjuternaja-mebel...,2024-06-05,Нет в наличии,77805,99.0,NaN
4399,Освежитель воздуха автоматический Air Wick Lif...,https://tiraet.com/catalog/Kompjuternaja-mebel...,2024-06-05,Нет в наличии,77803,99.0,NaN
6426,"Электрочайник Braun Multiquick 3 WK 300 (1.7л,...",https://tiraet.com/catalog/bytovaya/tehnika-dl...,2024-06-05,Нет в наличии,77716,NaN,NaN
6427,"Кофеварка Adler AD 4404cr (молотый кофе, эспре...",https://tiraet.com/catalog/bytovaya/tehnika-dl...,2024-06-05,Нет в наличии,75833,1558.0,NaN
8274,"Лампа светодиодная BELLIGHT R63 7,5W E27 4000K",https://tiraet.com/catalog/svet-i-jelektrika/l...,2024-06-05,Нет в наличии,78759,NaN,NaN
9151,"Точка доступа, настенная установка, Wi-Fi 6, T...",https://tiraet.com/catalog/setevoe-oborudovani...,2024-06-05,Нет в наличии,78432,NaN,NaN
9152,"PATCH CORD FTP CAT.5E, 20м. LANBERG (PCF5-10CC...",https://tiraet.com/catalog/setevoe-oborudovani...,2024-06-05,Нет в наличии,75701,6.5,7.0
11422,Картридж EPSON MX-80/LX-300/400/800/850/880 Pa...,https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Нет в наличии,249,38.0,NaN
11423,Комплект восстановления Canon IR-1435 CET5280...,https://tiraet.com/catalog/rashodnye-materialy...,2024-06-05,Нет в наличии,68689,594.0,NaN


In [126]:
df['counts'].unique()

array(['Много', 'Мало', 'Достаточно', 'Нет в наличии'], dtype=object)

In [ ]:
preprocessing(df)